In [28]:
import pandas as pd
import numpy as np
from utils import read_file, tokenize_latex
df = read_file("data/1601/*")


error with: data/1601/1601.06597.json


In [29]:
import re
def clean_exp(exp):
    exp = re.sub("\\\\begin{equation[^}]*}","" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\end{equation[^}]*}","" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\begin{split[^}]*}","\\\\begin{aligned}" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\end{split[^}]*}","\\\\end{aligned}" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\begin{gather[^}]*}","\\\\begin{aligned}" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\end{gather[^}]*}","\\\\end{aligned}" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\begin{align[^}]*}","\\\\begin{aligned}" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\end{align[^}]*}","\\\\end{aligned}" ,exp, flags=re.IGNORECASE)
    exp = re.sub("\\\\label{[^}]*}","" ,exp , flags=re.IGNORECASE)
    exp = re.sub("\\\\n", "",exp, flags=re.IGNORECASE)
    exp = re.sub("\\$", "",exp , flags=re.IGNORECASE)
    return exp

In [30]:
df["processed"] = df["text"].apply(lambda x: clean_exp(x)).apply(lambda x: tokenize_latex(x))

In [31]:
from gensim import corpora
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
equations = df["processed"].tolist()
dictionary = corpora.Dictionary(equations)

corpus = [dictionary.doc2bow(eq) for eq in equations]
corpora.MmCorpus.serialize('/tmp/equations.mm', corpus)
# our vector space model
print(corpus[1:5])

2017-12-02 00:49:54,164 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-12-02 00:49:54,691 : INFO : adding document #10000 to Dictionary(15769 unique tokens: [u'\\xi\\leq', u'4E+6F', u'0.000004', u'-P\\cdot\\frac', u'i|S']...)
2017-12-02 00:49:55,213 : INFO : adding document #20000 to Dictionary(25840 unique tokens: [u'\\xi\\leq', u'eF', u'q-1', u'q-2', u'+i+1']...)
2017-12-02 00:49:55,714 : INFO : adding document #30000 to Dictionary(35023 unique tokens: [u'\\xi\\leq', u'eF', u'-5AB\\sqrt', u'q-1', u'q-2']...)
2017-12-02 00:49:56,251 : INFO : adding document #40000 to Dictionary(43511 unique tokens: [u'\\xi\\leq', u'eF', u'-5AB\\sqrt', u'Model~III.~~~~~~~c', u'q-1']...)
2017-12-02 00:49:56,807 : INFO : adding document #50000 to Dictionary(53030 unique tokens: [u'\\xi\\leq', u'eF', u'-5AB\\sqrt', u'Model~III.~~~~~~~c', u'q-1']...)
2017-12-02 00:49:57,126 : INFO : built Dictionary(57545 unique tokens: [u'\\xi\\leq', u'eF', u'-5AB\\sqrt', u'Model~III.~~~~~~~c', u'q-1'

[[(5, 1), (6, 1), (8, 3), (12, 4), (13, 3), (14, 4), (15, 4), (17, 1), (18, 3), (19, 3), (21, 1), (22, 1), (23, 1)], [(0, 1), (3, 1), (5, 1), (6, 1), (8, 5), (9, 1), (10, 4), (12, 3), (13, 2), (14, 5), (15, 5), (17, 2), (18, 3), (19, 5), (20, 1), (22, 2), (24, 1), (25, 1), (26, 1)], [(2, 1), (4, 1), (5, 1), (6, 1), (12, 2), (14, 6), (15, 6), (17, 1), (19, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)], [(1, 1), (2, 1), (4, 1), (5, 3), (6, 3), (7, 3), (8, 1), (14, 6), (15, 6), (17, 1), (19, 2), (20, 2), (29, 1), (40, 1), (41, 2), (42, 1), (43, 2)]]


In [32]:
from gensim import corpora, models, similarities, matutils
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
from gensim import similarities
index = similarities.Similarity("index/", corpus_tfidf, len(dictionary.keys()), num_best = 100)

2017-12-02 00:50:09,937 : INFO : collecting document frequencies
2017-12-02 00:50:09,950 : INFO : PROGRESS: processing document #0
2017-12-02 00:50:10,032 : INFO : PROGRESS: processing document #10000
2017-12-02 00:50:10,099 : INFO : PROGRESS: processing document #20000
2017-12-02 00:50:10,168 : INFO : PROGRESS: processing document #30000
2017-12-02 00:50:10,247 : INFO : PROGRESS: processing document #40000
2017-12-02 00:50:10,350 : INFO : PROGRESS: processing document #50000
2017-12-02 00:50:10,406 : INFO : calculating IDF weights for 56379 documents and 57544 features (1222940 matrix non-zeros)
2017-12-02 00:50:10,461 : INFO : starting similarity index under index/
2017-12-02 00:50:17,575 : INFO : PROGRESS: fresh_shard size=10000
2017-12-02 00:50:24,426 : INFO : PROGRESS: fresh_shard size=20000
2017-12-02 00:50:28,874 : INFO : PROGRESS: fresh_shard size=30000
2017-12-02 00:50:30,080 : INFO : creating sparse index
2017-12-02 00:50:30,082 : INFO : creating sparse matrix from corpus
201

In [33]:
docs = ["".join(eq) for eq in df["processed"].tolist()]

In [34]:
docs

[u'\\Phi_{z}(L)=\\sum_{i=1}^{N}\\frac{1}{C_{i}\\timesV_{\\rmmax,i}},',
 u'V_{\\rmmax}={\\rmmin}(V_{\\rmmax,radio},V_{\\rmmax,optical})',
 u'V_{\\rmmax,radio}=\\sum\\limits_{i=1}^{n}F_i\\timesV_{\\rmmax,radio,i}(z_{\\rmmax,i}),',
 u'C_{R}=0.5\\left[1+{\\rmerf}\\left(\\frac{{\\rmSNR}-\\mu}{\\sqrt{2}\\sigma}\\right)\\right]',
 u'\\Phi(L)=\\frac{C}{(L_{*}/L)^{A}+(L_{*}/L)^{B}},',
 u'\\Phi_{z}(L)=(1+z)^{k}\\Phi_{0}(L),',
 u'\\Phi_{z}(L)=\\Phi_{0}\\Bigg(\\frac{L}{(1+z)^{k}}\\Bigg),',
 u'\\log_{10}\\Phi_{z}=m\\log_{10}L+k\\log_{10}(1+z)+c,',
 u'\\log_{10}\\Phi_{z}=m(\\log_{10}(L)-k\\log_{10}(1+z))+c.',
 u'\\Gamma=\\gamma^{-1}(1-\\beta\\cos\\theta)^{-1},',
 u'\\sigma_{\\rmmNWA}=\\int_{M-\\Delta}^{M+\\Delta}dm_{{\\gamma\\gamma}}\\bigg[\\frac{d\\sigma_{\\rmsig}}{dm_{{\\gamma\\gamma}}}\\bigg]_{\\rmpeak},\\quad\\hbox{forabroadresonance}.',
 u'K_{\\rmintf}=\\frac{\\sigma_{\\rmmNWA}}{\\sigma_{\\rmprod}\\cdot{\\rmBr}_{{\\gamma\\gamma}}}',
 u'\\chi^2_{\\rmSM}=7.02,\\,4.93,\\,17.77,\\,1.52,\\,16.65,',


In [35]:
import json
json.dump(docs, open("formula_list.json", "w+"))

In [7]:
class Query:
    '''
    The important thing here is that the labels on the data coincide with the labels on the columns.
    '''
    def __init__(self, index, docs, dictionary, tokenize_latex, cols):
        self.index = index
        self.docs = docs
        self.dictionary = dictionary
        self.tokenize_latex = tokenize_latex
        self.columns = cols
    def _convert_query(self, query):
        query = self.dictionary.doc2bow(self.tokenize_latex(query))
        sims = self.index[query]
        neighbors = sorted(sims, key=lambda item: -item[1])
        neighbors = {"neighbors":[{self.columns[0]: {"data": self.docs[n[0]], "fmt": "math"}, self.columns[1]: {"data": float(n[1])}} for n in neighbors]} if neighbors else {"neighbors": []}
        return neighbors
    def query(self, q):
        return self._convert_query(q)
        



In [8]:
index

In [9]:
dictionary

In [10]:
q = Query(index, docs, dictionary, tokenize_latex, ["neighbor", "similarity_score"])
q.query("\\frac")

2017-12-01 11:17:28,732 : INFO : creating matrix with 4 documents and 44 features
2017-12-01 11:17:28,736 : INFO : creating dense shard #0
2017-12-01 11:17:28,741 : INFO : saving index shard to index/.0
2017-12-01 11:17:28,745 : INFO : saving MatrixSimilarity object under index/.0, separately None
2017-12-01 11:17:28,748 : INFO : saved index/.0
2017-12-01 11:17:28,751 : INFO : loading MatrixSimilarity object from index/.0
2017-12-01 11:17:28,752 : INFO : loaded index/.0


{'neighbors': [{'neighbor': {'data': u'\\begin{aligned}\\min_{L,S}\\sum_{m=1}^{M}\\sum_{i=1}^{N_{m}}&\\frac{1}{2}[max(0,1-Y_{m}^{i}(Ls^{m})^{T}X_{m}^{i})]^{2}\\\\&+\\gamma\\|L\\|_{1}+\\lambda\\|L\\|^{2}_{F}\\\\\\end{aligned}',
    'fmt': 'math'},
   'similarity_score': {'data': 0.0750708281993866}},
  {'neighbor': {'data': u'\\begin{aligned}\\min_{L,S}\\sum_{m=1}^{M}\\sum_{i=1}^{N_{m}}&\\frac{1}{2}[max(0,1-Y_{m}^{i}(Ls^{m})^{T}X_{m}^{i})]^{2}\\\\&+\\mu\\sum_{k=1}^{K}\\sum_{g=1}^{G}\\|s_{k}^{g}\\|_{2}\\\\\\end{aligned}',
    'fmt': 'math'},
   'similarity_score': {'data': 0.061607733368873596}},
  {'neighbor': {'data': u'\\begin{aligned}\\min_{L,S}\\sum_{m=1}^{M}\\sum_{i=1}^{N_{m}}&\\frac{1}{2}[max(0,1-Y_{m}^{i}(Ls^{m})^{T}X_{m}^{i})]^{2}\\\\&+\\mu\\sum_{k=1}^{K}\\sum_{g=1}^{G}\\|s_{k}^{g}\\|_{2}+\\gamma\\|L\\|_{1}+\\lambda\\|L\\|^{2}_{F}\\\\\\end{aligned}',
    'fmt': 'math'},
   'similarity_score': {'data': 0.052263256162405014}}]}

In [11]:
from mathviz_hopper.src.table import Table
t = Table(q)
t.print_ipython()

# TODO: lambda function to convert each column
# Schema:
# also, enable non-autocomplete selections (the models should be able to handle this)
# also, add a warning if the settings are still loading, and other notifications.
# Data: [{Column A: {name: A, index: [], filter_func: lambda x : f(x)}} , {Column B: {name: B, index: [], filter_func: lambda x : f(x)}} 

Bottle v0.13-dev server starting up (using MyWSGIRefServer())...
Listening on http://localhost:8081/
Hit Ctrl-C to quit.



127.0.0.1 - - [01/Dec/2017 11:17:30] "GET /settings HTTP/1.1" 200 1007


In [12]:
import requests
r = requests.post('http://localhost:8081/query', json={"query": "\\begin{equation}\n\\Phi_{z}(L) = \\sum_{i=1}^{N} \\frac{1}{C_{i} \\times V_{\\rm max, i}} ,\n\\label{EQ1}\n\\end{equation}\n"})

127.0.0.1 - - [01/Dec/2017 11:17:30] "POST /query HTTP/1.1" 200 915


In [13]:
r.json()

{u'neighbors': [{u'neighbor': {u'data': u'\\begin{aligned}\\min_{L,S}\\sum_{m=1}^{M}\\sum_{i=1}^{N_{m}}&\\frac{1}{2}[max(0,1-Y_{m}^{i}(Ls^{m})^{T}X_{m}^{i})]^{2}\\\\&+\\gamma\\|L\\|_{1}+\\lambda\\|L\\|^{2}_{F}\\\\\\end{aligned}',
    u'fmt': u'math'},
   u'similarity_score': {u'data': 0.2967531681060791}},
  {u'neighbor': {u'data': u'\\begin{aligned}\\min_{L,S}\\sum_{m=1}^{M}\\sum_{i=1}^{N_{m}}&\\frac{1}{2}[max(0,1-Y_{m}^{i}(Ls^{m})^{T}X_{m}^{i})]^{2}\\\\&+\\mu\\sum_{k=1}^{K}\\sum_{g=1}^{G}\\|s_{k}^{g}\\|_{2}\\\\\\end{aligned}',
    u'fmt': u'math'},
   u'similarity_score': {u'data': 0.29066944122314453}},
  {u'neighbor': {u'data': u'\\begin{aligned}\\min_{L,S}\\sum_{m=1}^{M}\\sum_{i=1}^{N_{m}}&\\frac{1}{2}[max(0,1-Y_{m}^{i}(Ls^{m})^{T}X_{m}^{i})]^{2}\\\\&+\\mu\\sum_{k=1}^{K}\\sum_{g=1}^{G}\\|s_{k}^{g}\\|_{2}+\\gamma\\|L\\|_{1}+\\lambda\\|L\\|^{2}_{F}\\\\\\end{aligned}',
    u'fmt': u'math'},
   u'similarity_score': {u'data': 0.2799034118652344}}]}

In [14]:
import requests
r = requests.get('http://localhost:8081/settings')

127.0.0.1 - - [30/Nov/2017 14:03:54] "GET /settings HTTP/1.1" 200 1007


In [15]:
r.json()

{u'columns': [{u'Header': u'neighbor', u'accessor': u'neighbor'},
  {u'Header': u'similarity_score', u'accessor': u'similarity_score'}],
 u'docs': {u'\\': {u'b': {u'e': {u'g': {u'i': {u'n': {u'{': {u'a': {u'l': {u'i': {u'g': {u'n': {u'e': {u'd': {u'}': {u'\\': {u'm': {u'i': {u'n': {u'_': {u'{': {},
                      u'{L,S}\\sum_{m=1}^{M}\\sum_{i=1}^{N_{m}}&\\frac{1}{2}[max(0,1-Y_{m}^{i}(Ls^{m})^{T}X_{m}^{i})]^{2}\\\\&+\\gamma\\|L\\|_{1}+\\lambda\\|L\\|^{2}_{F}\\\\\\end{aligned}': {u'full_word': 1},
                      u'{L,S}\\sum_{m=1}^{M}\\sum_{i=1}^{N_{m}}&\\frac{1}{2}[max(0,1-Y_{m}^{i}(Ls^{m})^{T}X_{m}^{i})]^{2}\\\\&+\\mu\\sum_{k=1}^{K}\\sum_{g=1}^{G}\\|s_{k}^{g}\\|_{2}+\\gamma\\|L\\|_{1}+\\lambda\\|L\\|^{2}_{F}\\\\\\end{aligned}': {u'full_word': 1},
                      u'{L,S}\\sum_{m=1}^{M}\\sum_{i=1}^{N_{m}}&\\frac{1}{2}[max(0,1-Y_{m}^{i}(Ls^{m})^{T}X_{m}^{i})]^{2}\\\\&+\\mu\\sum_{k=1}^{K}\\sum_{g=1}^{G}\\|s_{k}^{g}\\|_{2}\\\\\\end{aligned}': {u'full_word': 1}}}}}},
   

In [16]:

def insert(st, trie):
    i = 0
    for s in st:
        if s not in trie.keys(): 
            trie[s] = {}
        if i == 20:
            break
        trie = trie[s]
        i+=1
        
    if i == 20: 
        trie[st[i:]] = {}
        trie = trie[st[i:]]
    trie["full_word"] = 1
        
    
def construct_trie(list_of_str):
    trie = {}
    for st in list_of_str:
        insert(st, trie)
    return trie
    

In [17]:
docs_trie = construct_trie(clean_docs)


NameError: name 'clean_docs' is not defined

127.0.0.1 - - [30/Nov/2017 14:03:54] "GET /settings HTTP/1.1" 200 1007
127.0.0.1 - - [30/Nov/2017 14:04:09] "POST /query HTTP/1.1" 200 916


In [ ]:

opts = {
    "columns": [
            {
                "Header": "Word",
                "accessor": "word"
            },
            {
                "Header": "Similarity",
                "accessor": "sim"
            }
   ],
    "port": "8081",
    "docs": docs_trie
}

In [ ]:
import json
with open("mathviz_hopper/webpage/mathviz-js-components/public/settings.json", "w+") as f:
    json.dump(opts, f)

In [ ]:
def word_exists(trie, word):
    
    for i, s in enumerate(word):        
        if s in trie.keys(): 
            trie = trie[s]
        else: return False
            
        if i == 19: 
            break
            
    if i == 19:
        s = word[i+1:]
        if s not in trie.keys(): return False
        trie = trie[s]

    return True if 'full_word' in trie.keys() else False
        
        

In [ ]:
word_exists(trie, "fdakfkajshdfjkahsdfkjhadfdfdfdffdfdsdjkfhaskj")

In [ ]:
docs = ["".join(eq) for eq in df["processed"].tolist()]
docs_trie = construct_trie(docs)